# Imports

In [1]:
import numpy as np
import pandas as pd
import sys
libraries = (('Numpy', np), ('Pandas', pd))

print("Python Version:", sys.version, '\n')
for lib in libraries:
    print('{0} Version: {1}'.format(lib[0], lib[1].__version__))

Python Version: 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0] 

Numpy Version: 1.16.4
Pandas Version: 0.23.0


In [2]:
import txtFiles as tf
import ftCommands as ftc
import evaluatePredictions as ep

sys.path.append('../myModules/')
import kleptoFunctions as kf

# Data

In [3]:
data_path = 'data/'

In [4]:
data = kf.puking_file('2019.06.25_cleaned_newsgroups', data_path)

Filename: 2019.06.25_cleaned_newsgroups 
# of Folders: 5 
Type: <class 'pandas.core.frame.DataFrame'> 
Len: 18752


# FastText Txt files

## split ids

In [5]:
random_state = 19

In [6]:
set_ids = tf.training_validation_holdout_split(data['_id'], random_state)

holdout	 3751
training	 11250
validation	 3751


## txt files

In [7]:
for name_part in set_ids.keys():
    filepath = data_path+'%s.txt' % name_part
    df = data[data['_id'].isin(set_ids[name_part])]
    tf.make_txtfile(filepath, df, 'tier2_targets')
    print(filepath)

data/holdout_19.txt
data/T1_training_19.txt
data/T1_validation_19.txt


In [8]:
train_filename = [k for k in set_ids.keys() if 'training' in k][0]+'.txt'
train_address = data_path+train_filename
test_filename = [k for k in set_ids.keys() if 'validation' in k][0]+'.txt'
test_address = data_path+test_filename
holdout_filename = [k for k in set_ids.keys() if 'holdout' in k][0]+'.txt'
holdout_address = data_path+holdout_filename

# Training Model

In [9]:
# FastText Parameters
ngram = '2'
lr = '0.5'
dim = '200'
ws = '5'
epoch = '25'
loss = 'ns'

In [10]:
model_address, predict_address = ftc.train_test_predict(train_address, test_address, ngram, lr, dim, ws, epoch, loss)
print(model_address)
print(predict_address)

N	3751
P@1	0.838
R@1	0.838

data/T1_model
data/T1_prediction_19.txt


# Using Model

In [11]:
predict_address = ftc.test_predict(holdout_address, model_address)
print(predict_address)

N	3751
P@1	0.848
R@1	0.848

data/holdout_prediction_19.txt


In [12]:
# how did it do on our holdout set?
precision, recall, fscore, support = ep.score_txtfiles(holdout_address, predict_address)

precision: 0.8444944232670307
recall: 0.8421878952390142
fscore: 0.8417933346554742
